In [1]:
import pandas as pd
import eurostat
import numpy as np

# Education Attainment


In [2]:
# Import the data from Eurostat
df = eurostat.get_data_df('edat_lfse_04')
df.columns = df.columns.astype(str)

# Drop all years before 2012 and columns unit and age (same for all entries)
df = df.drop(df.loc[:, '2011': '2000'].columns, axis = 1)
df = df.drop(['unit', 'age'], axis = 1)

# Keep only total values and discard the gender column
df = df[df.sex == 'T']
df = df.drop('sex', axis = 1)

# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
education_attainment = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = education_attainment.loc[education_attainment['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow (Metro area) is approx 33% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
education_attainment[education_attainment['City'] == 'Glasgow'] = Glasgow


# Impute the missing values from 2012 using padding
education_attainment = education_attainment.interpolate(method='pad',axis=1)

# Split in to 3 separate tables depending on education level
edu_ED_0_2 = education_attainment[education_attainment['isced11'] == 'ED0-2']
edu_ED_3_4 = education_attainment[education_attainment['isced11'] == 'ED3_4']
edu_ED_5_8 = education_attainment[education_attainment['isced11'] == 'ED5-8']

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
education_attainment  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= edu_ED_0_2[ list(edu_ED_0_2.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Education Attainment ED 0-2"})
    yearly_data.insert(6, "Education Attainment ED 3-4", edu_ED_3_4[f"{year}"].to_numpy())
    yearly_data.insert(7, "Education Attainment ED 5-8", edu_ED_5_8[f"{year}"].to_numpy())
    education_attainment = education_attainment.append(yearly_data)
    
education_attainment = education_attainment.reset_index(drop=True)

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


# Victims in Road Accidents

In [3]:
# Get the data from Eurostat
df = eurostat.get_data_df('tran_r_acci')
df.columns = df.columns.astype(str)
# Drop all years before 2012, keep only with unit measure Per Million Inhabitants and accident type deadly
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
df = df[df.unit == 'P_MHAB']
df = df[df.victim == 'KIL']

# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Insert missing cloumn for 2019
df.insert(3,"2019",np.NaN)

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
road_accidents = pd.merge(target_cities, df, on=[' NUTS 2'])

# Check for missing cities
missing_cities = target_cities[-target_cities[' NUTS 2'].isin(road_accidents[' NUTS 2'])]


print("Missing cities: ", missing_cities.values[:,0])

# Data from Icelandic ministry of transportation
#https://www.samgongustofa.is/umferd/tolfraedi/slysatolur/arsskyrslur-slysaskraningar/
# Reported as deaths per 10,000
Reykjavik  = [0.0, 30.0, 20.0, 10.0, 20.0, 0.0, 10.0, 10.0]
Reykjavik = ['Reykjavik', 'IS001C1','IS00','IS','KIL','P_MHAB']+Reykjavik
road_accidents.loc[77] = Reykjavik

# Table 1-1 in : https://www.abs.gov.rs/admin/upload/documents/20181016102533-statistical_report_2016_english.pdf
# 20% of the population is in Belgrade so we take 20% of the accidents as occuring there
# and divide by 1.7 (million inhabitants)
Belgrade = [np.NaN, np.NaN,np.NaN, 619, 594, 476, 548, 551]
Belgrade = np.multiply(Belgrade,(0.2/1.7))
Belgrade = np.around(Belgrade, 1)
Belgrade = ['Belgrade', '-','RS11','RS','KIL','P_MHAB']+Belgrade.tolist()
road_accidents.loc[78] = Belgrade

# Traffic death data from Scotland
# https://statistics.gov.scot/data/road-safety
# Select by region

# Divide by 0.5 (million inhabitants)
Edinburgh = [np.NaN, 5, 6, 9, 3, 11, 8, 13]
Edinburgh = np.divide(Edinburgh,0.5)
Edinburgh = np.around(Edinburgh, 1)
Edinburgh = ['Edinburgh', 'UK007C1','UKM7','UK','KIL','P_MHAB']+Edinburgh.tolist()
road_accidents.loc[79] = Edinburgh

# Divide by 0.6 (million inhabitants)
Glasgow = [np.NaN, 10, 7, 8, 15, 18, 4, 7]
Glasgow = np.divide(Glasgow,0.6)
Glasgow = np.around(Glasgow, 1)
Glasgow = ['Glasgow', 'UK004C1','UKM3','UK','KIL','P_MHAB']+Glasgow.tolist()
road_accidents.loc[80] = Glasgow

# Impute using padding to fill missing values
road_accidents.loc[:, '2019':'2013'] = road_accidents.loc[:, '2019':'2012'].fillna(method='backfill',axis=1)
road_accidents.loc[:, '2019':'2013'] = road_accidents.loc[:, '2019':'2012'].fillna(method='ffill',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
deaths_in_road_accidents  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= road_accidents[ list(road_accidents.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Deaths_in_road_accidents"})
    deaths_in_road_accidents = deaths_in_road_accidents.append(yearly_data)
deaths_in_road_accidents = deaths_in_road_accidents.reset_index(drop=True)

Missing cities:  ['Belgrade' 'Edinburgh' 'Glasgow' 'Reykjavik']


# Average Working Hours

In [4]:
# Load the datasheet from eurostat
df = eurostat.get_data_df('lfst_r_lfe2ehour')

# Drop all years before 2012 and columns unit and age (same for all entries)
df.columns = df.columns.astype(str)
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Extract the data for both sexes (Total) and age group 15-74
df = df[df.sex == 'T']
df = df.drop('sex', axis = 1)
df = df[df.age == 'Y15-74']

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
avg_hours = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = avg_hours.loc[avg_hours['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow is 40% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
avg_hours[avg_hours['City'] == 'Glasgow'] = Glasgow

# Impute the rest using padding
avg_hours = avg_hours.interpolate(method='pad',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
avg_working_hours  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= avg_hours[ list(avg_hours.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Average Working Hours"})
    avg_working_hours = avg_working_hours.append(yearly_data)
avg_working_hours = avg_working_hours.reset_index(drop=True)

# Unemployment Rate

In [5]:
# Read the datasheet from Eurostat
df = eurostat.get_data_df('lfst_r_lfu3rt')

# Drop all years before 2012 and columns unit and age (same for all entries)
df.columns = df.columns.astype(str)
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Extract the data for both sexes (Total) and age group 15-74
df = df[df.sex == 'T']
df = df[df.age == 'Y15-74']
df = df.drop(['sex','unit','age'], axis = 1)

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
unemployment_rate = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = unemployment_rate.loc[unemployment_rate['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow is 40% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
unemployment_rate[unemployment_rate['City'] == 'Glasgow'] = Glasgow

# Impute the rest using padding
unemployment_rate = unemployment_rate.interpolate(method='pad',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
unemployment_rate_data  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= unemployment_rate[ list(unemployment_rate.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Unemployment_Rate"})
    unemployment_rate_data = unemployment_rate_data.append(yearly_data)
unemployment_rate_data = unemployment_rate_data.reset_index(drop=True)

# Merging of the datasets

In [9]:
# Code if we want to do it with csv files, requires all in the same format and not other existing csv files in the folder

#initial = pd.read_csv('Education_Attainment.csv')
#merged = initial
#
#for file_name in glob.glob('*.csv'):
#    if file_name == 'Education_Attainment.csv':
#        continue 
#    temp_dataframe = pd.read_csv(file_name)
#    values = temp_dataframe.drop(columns=[' City Code', ' NUTS 2',' Country'])
#    merged = merged.merge(values, on=['City', 'Year'])


# Code to do it directly in the notebook without saving to csv files first
# Right now requires all data to be complete
data_sheets = [deaths_in_road_accidents,avg_working_hours,unemployment_rate_data]

initial = education_attainment
merged = initial

for file_name in data_sheets:
    temp_dataframe = file_name
    values = temp_dataframe.drop(columns=[' City Code', ' NUTS 2',' Country'])
    merged = merged.merge(values, on=['City', 'Year'])
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
    
merged

,City,City Code,NUTS 2,Country,Year,Education Attainment ED 0-2,Education Attainment ED 3-4,Education Attainment ED 5-8,Deaths_in_road_accidents,Average Working Hours,Unemployment_Rate
0,Amsterdam,NL002C1,NL32,NL,2012,23.4,38.0,38.5,33.0,30.6,5.4
1,Ankara,TR001C1,TR51,TR,2012,52.4,22.1,25.5,42.0,48.7,8.3
2,Antwerp,BE002C1,BE2,BE,2012,26.1,38.5,35.4,60.0,37.2,4.5
3,Athens,EL001C1,EL30,EL,2012,22.5,44.1,33.4,59.0,40.6,25.8
4,Barcelona,ES002C1,ES51,ES,2012,44.9,22.3,32.8,45.0,37.8,22.5
5,Valencia,ES003C1,ES51,ES,2012,44.9,22.3,32.8,45.0,37.8,22.5
6,Belfast,UK012C1,UKN0,UK,2012,29.5,38.4,32.1,26.0,36.2,7.4
7,Belgrade,-,RS11,RS,2012,11.5,57.1,31.4,64.8,42.0,17.9
8,Bergen,NO002C1,NO05,NO,2012,17.1,46.2,36.6,29.0,33.8,2.9
9,Berlin,DE001C1,DE30,DE,2012,14.8,48.0,37.2,13.0,35.8,10.4
